In [ ]:
%matplotlib inline
# # Smooth pursuit
# This is an idealized model of the smooth pursuit reflex, including two ocular muscles, a moving visual stimulus and spiking neural control.
from brian2 import *
import ipywidgets as ipw

In [ ]:
progress_slider = ipw.FloatProgress(description="Simulation progress", min=0, max=1)
progress_slider.layout.width = '95%'
progress_slider.style = {'description_width': '30%'}
def update_progress(elapsed, complete, t_start, duration):
    progress_slider.value = complete
    if complete==1:
        progress_slider.bar_style = 'success'
        progress_slider.description = "Simulation complete"
    elif complete==0:
        progress_slider.bar_style = ''
        progress_slider.description = "Simulation started"
    else:
        progress_slider.bar_style = ''
        time_remaining = elapsed/complete-elapsed
        seconds_remaining = int(time_remaining)
        minutes_remaining = seconds_remaining//60
        seconds_remaining -= 60*minutes_remaining
        hours_remaining = minutes_remaining//60
        minutes_remaining -= 60*hours_remaining
        remaining = str(seconds_remaining)+'s'
        if minutes_remaining or hours_remaining:
            remaining = str(minutes_remaining)+'m '+remaining
        if hours_remaining:
            remaining = str(hours_remaining) + 'h ' + remaining
        progress_slider.description = "Simulating (%s remaining)" % remaining

In [ ]:
def runsim(characteristic_relaxation_time_ms):
    characteristic_relaxation_time = characteristic_relaxation_time_ms*ms
    seed(79620)
    # Each of the two antagonistic ocular muscles is modelled as a spring of elasticity k and some friction. To simplify, we consider that the eye moves laterally, rather than rotates. If x is the position of the eye with 0 being the center, then the lengths of the springs are $L+x$ and $L-x$. The dynamics of the eye is then:
    # $$ m\frac{d^2x}{dt^2} = - k((L+x)-x_L) + k((L-x)-x_R) - f.\frac{dx}{dt}$$
    # or:
    # $$ m\frac{d^2x}{dt^2} = k(x_L-x_R-2x) - f.\frac{dx}{dt}$$
    #
    # We see that $x_0=(x_L-x_R)/2$ is the equilibrium position of the eye. Here we have assumed that spring elasticities are identical. We can rewrite with just two parameters $\alpha$ and $\beta$:
    # $$ \frac{d^2x}{dt^2} = \alpha(x_0-x) - \beta.\frac{dx}{dt}$$
    #
    # We will assume that eye position can move between -1 and 1.
    #
    # We consider that the resting length is the variable on which motoneurons act. Each spike from a motoneuron produces a waveform of contraction ("twitch"), i.e., a change in resting length. We consider that contractions add linearly, and resting lengths relax exponentially. By linearity it follows that we can simply express the action of motoneurons on the equilibrium position of the eye $x_0$, which relaxes exponentially to the center position 0.
    #
    # Finally, we consider a visual object that oscillates in the visual field.
    alpha = (1/(characteristic_relaxation_time))**2 # characteristic relaxation time is 50 ms
    beta = 1/(characteristic_relaxation_time) # friction parameter
    tau_muscle = 20*ms # relaxation time of muscle contraction
    tau_object = 500*ms # time constant of object movement

    eqs_eye = '''
    dx/dt = velocity : 1
    dvelocity/dt = alpha*(x0-x)-beta*velocity : 1/second
    dx0/dt = -x0/tau_muscle : 1
    dx_object/dt = (noise - x_object)/tau_object:  1
    dnoise/dt = -noise/tau_object + tau_object**-0.5*xi : 1
    '''
    eye = NeuronGroup(1, model=eqs_eye, method='euler', name='eye')

    # We now define two motoneurons, one for each muscle
    taum = 20*ms
    motoneurons = NeuronGroup(2, model= 'dv/dt = -v/taum : 1', threshold = 'v>1',
                              reset = 'v=0', refractory = 5*ms, method='exact',
                              name='motoneurons')

    # The motoneurons project to the eye, and each spike produces a small contraction.
    motosynapses = Synapses(motoneurons, eye, model = 'w : 1', on_pre = 'x0+=w', name='moto_synapses')
    motosynapses.connect() # connects all motoneurons to the eye
    motosynapses.w = [-0.5,0.5]

    # For sensory neurons, we simplify by considering spiking neurons which directly respond to light, ie they represent both photoreceptors and retinal ganglion cells.
    N = 20
    width = 2./N # width of receptive field
    gain = 4.
    eqs_retina = '''
    I = gain*exp(-((x_object-x_eye-x_neuron)/width)**2) : 1
    x_neuron : 1 (constant)
    x_object : 1 (linked) # position of the object
    x_eye : 1 (linked) # position of the eye
    dv/dt = (I-(1+gs)*v)/taum : 1
    gs : 1 # total synaptic conductance
    '''
    retina = NeuronGroup(N, model = eqs_retina, threshold = 'v>1', reset = 'v=0', method='exact', name='retina')
    retina.v = 'rand()'
    retina.x_eye = linked_var(eye, 'x')
    retina.x_object = linked_var(eye, 'x_object')
    retina.x_neuron = '-1.0 + 2.0*i/(N-1)'

    # Finally we connect sensory neurons to motoneurons. Sensory neurons on each hemifield connects to the corresponding motoneuron, with a strength that scales with eccentricity.
    sensorimotor_synapses = Synapses(retina, motoneurons, model = 'w : 1 (constant)', on_pre = 'v+=w', name='retinal_synapses')
    sensorimotor_synapses.connect(j = 'int(x_neuron_pre > 0)')
    sensorimotor_synapses.w = '20*abs(x_neuron_pre)/N_pre'

    # We record the position of the eye, of the object, and spikes produced by the retina and motoneurons.
    M = StateMonitor(eye, ('x', 'x0', 'x_object'), record = True, name='mon_eye')
    S_retina = SpikeMonitor(retina, name='mon_retina')
    S_motoneurons = SpikeMonitor(motoneurons, name='mon_moto')

    run(10*second, report=update_progress, report_period=1*second)
    plt.style.use('paper_mplstyle.cfg')
    plt.figure(figsize=(6, 6*0.75))
    plt.subplot2grid((3, 1),(0, 0), rowspan=2)
    h0, = plt.plot(S_retina.t/second,S_retina.i,'|k', markeredgecolor='k', label='retina')
    motoneuron_spikes = S_motoneurons.spike_trains()
    h1, = plt.plot(motoneuron_spikes[0]/second, np.ones(S_motoneurons.count[0])*N, '|', color='C0',
    markeredgecolor='C0', label='left motoneuron')
    h2, = plt.plot(motoneuron_spikes[1]/second, np.ones(S_motoneurons.count[1])*(N+1), '|', color='C1',
    markeredgecolor='C1', label='right motoneuron')
    plt.yticks([])
    plt.ylabel('neuron index')
    plt.gca().spines['bottom'].set_visible(False)
    plt.xticks([])

    plt.subplot2grid((3, 1),(2, 0))
    plt.axhline(0, color='gray')
    plt.plot(M.t/second, M.x[0],'k', label='eye')
    #plot(M.t/ms, M.x0[0],'b')
    plt.plot(M.t/second, M.x_object[0],color='C2', label='object')
    plt.yticks([-1, 1], ['left', 'right'])
    plt.xlabel('time (s)')
    plt.legend(loc='upper right', bbox_to_anchor=(1.0, 2.0))
    plt.tight_layout()
    
#runsim()
params = dict(
    characteristic_relaxation_time_ms=ipw.FloatSlider(description="Characteristic relaxation time (ms)",
                                                      min=1, max=100, step=1, value=50),
    )
for widget in params.values():
    widget.layout.width = '95%'
    widget.style = {'description_width': '30%'}
    
i = ipw.interactive(runsim, dict(manual=True, manual_name="Run simulation"), **params)
display(ipw.VBox(children=[i, progress_slider]))